In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

import pandas as pd
import random
import csv
import os
import math
import numpy as np

from numba import jit
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.mode.chained_assignment = None

from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif

In [ ]:
# read in the dataframe you want to clean, can be a csv, hdf
traindf = pd.read_csv("./data/training_set_VU_DM.csv")

traindf = traindf[:10000]

# put filename here: we use hdf because it can store the entire dataset, while pickle files can't
filename = "./data/traindf_clean.hdf"

# if testing set is True we will not add the importance column (because we can't)
testing_set = False

In [ ]:
def competitors(df):
    """
    Make a new column in the dataframe (competitor_bool) for when there 
    exists a competitor and there are available rooms.
    1 is True, 0 is False.
    """

    # we say that there is no competitor with a lower price
    df["competitor_lower"] = 0 #competitor_bools
    
    # comp1rate = 1 if price is lower
    df["competitor_lower"][df["comp1_rate"] == 1] = 1
    df["competitor_lower"][df["comp2_rate"] == 1] = 1
    df["competitor_lower"][df["comp3_rate"] == 1] = 1
    df["competitor_lower"][df["comp4_rate"] == 1] = 1
    df["competitor_lower"][df["comp5_rate"] == 1] = 1
    df["competitor_lower"][df["comp6_rate"] == 1] = 1
    df["competitor_lower"][df["comp7_rate"] == 1] = 1
    df["competitor_lower"][df["comp8_rate"] == 1] = 1
    
    # we say at first there is no competitor hotel available
    df["competitor_available"] = 0
    
    # availability bool = 1 if there if the competitor and expedia are available
    df["competitor_available"][df["comp1_inv"] == 1] = 1
    df["competitor_available"][df["comp2_inv"] == 1] = 1
    df["competitor_available"][df["comp3_inv"] == 1] = 1
    df["competitor_available"][df["comp4_inv"] == 1] = 1
    df["competitor_available"][df["comp5_inv"] == 1] = 1
    df["competitor_available"][df["comp6_inv"] == 1] = 1
    df["competitor_available"][df["comp7_inv"] == 1] = 1
    df["competitor_available"][df["comp8_inv"] == 1] = 1

    return df


In [ ]:
def visitor_history(df):
    """
    Add column that tells us whether someone has visited a hotel before.
    Column name = total_visited; 1 is True, 0 is False.
    """
    
    # most visitors haven't visited a hotel yet
    df["visited_before"] = 0
    
    # where there is a history field filled in, visited_before is turned into 21
    df["visited_before"][df["visitor_hist_starrating"].notna() | df["visitor_hist_adr_usd"].notna()] = 1
    
    return df

In [ ]:
def price_category(df):
    """
    Add a column of categories of price_usd and a column
    that corrected price for number of nights.
    Preprocessing of quantile cut showed that categories are:
    [(6.0889999999999995, 69.0] < (69.0, 90.0] < (90.0, 110.0] 
    < (110.0, 136.0] < (136.0, 170.077] < (170.077, 239.0] < (239.0, 554655.0]]
    
    """
    
    df["avg_price_propid"] = None
    df["std_avg_price_propid"] = None
    
    for prop_id, group in df.groupby(["prop_id"]):      
        df["avg_price_propid"][df["prop_id"] == prop_id] = group["price_usd"].mean()
        df["std_avg_price_propid"][df["prop_id"] == prop_id] = group["price_usd"].std()
        
    df["price_correction"] = df["price_usd"]
    
    print("Correcting ", len(df["std_avg_price_propid"] > 50), " property prices")
    df["price_correction"][df["std_avg_price_propid"] > 50] = df["price_usd"] / df["srch_length_of_stay"]
                      
    # Correct for number of nights ad add as new column
#     df["price_correction"] = df["price_usd"] / df["srch_length_of_stay"]
    
    # Replace missing values
#     df["price_correction"].fillna((df["price_correction"].median()), inplace=True)
#     df["price_usd"].fillna((df["price_usd"].median()), inplace=True)
    
    # Make a new column of price categories
    # TODO: apparently this is not a number
#     df['PriceBand'] = pd.qcut(df["price_correction"], 7, labels=np.arange(1,8))
    
    return df



In [ ]:
before = price_category(traindf)

In [ ]:
after = probeerdf.copy()
for prop_id, group in df2.groupby(["prop_id"]):
    after["avg_price_propid"][after["prop_id"] == prop_id] = group["price_correction"].mean()
    after["std_avg_price_propid"][after["prop_id"] == prop_id] = group["price_correction"].std()

In [ ]:
print("Average standard deviation before correction:", before["std_avg_price_propid"].mean())
print("Average standard deviation after correction:", after["std_avg_price_propid"].mean())

print(len(before))
print(len(before["std_avg_price_propid"]))
print(len(before["std_avg_price_propid"].notna()))
display(before["price_usd"].describe())
display(before["std_avg_price_propid"].describe())

plt.figure()
plt.title("Standard deviation of mean property prices")
before["std_avg_price_propid"].hist(bins=100)

plt.figure()
plt.title("Difference between prices before and after correction")
after["price_usd_diff"] = abs(before["price_usd"] - before["price_correction"])
after["price_usd_diff"][after["price_usd_diff"] > 1].hist(bins=100)

In [ ]:
def price_quality(df):
    """
    Add a column of ratio price/quality to the DataFrame.
    """
    
    df["price_quality"] = None
    
    df["price_quality"][df["price_usd"].notna() & df["prop_starrating"].notna() & df["prop_starrating"] != 0] = df["price_usd"] / df["prop_starrating"]

    # Replace missing values with median
    df["price_quality"].fillna((df["price_quality"].median()), inplace=True)
    
    return df 

In [ ]:
def process_remaining_cols(df):
    """
    Add some remaining (and interesting columns) to the dataframe.
    """
    
    # Replace missing values with median
    df["prop_brand_bool"].fillna((df["prop_brand_bool"].median()), inplace=True)
    df["random_bool"].fillna((df["random_bool"].median()), inplace=True)
    
    # Boolians
    df["prop_location_score1"].fillna((df["prop_location_score1"].mean()), inplace=True)
    df["prop_location_score2"].fillna((df["prop_location_score2"].mean()), inplace=True)
        
     # Replace missing values with mean
    df["promotion_flag"].fillna((df["promotion_flag"].mean()), inplace=True)
    
    return df

In [ ]:
def add_score(df):
    """
    Add a score 
    """
    
    # every hotel that is clicked on gets an importance score of 1
    df["importance"] = df["click_bool"]
    
    # every hotel that is booked gets an importance score of 5 
    df["importance"][df["booking_bool"] == 1] = 5
        
    return df


def preprocessing(traindf, dollarprice=True):
    traindf = competitors(traindf)
    print("cleaned competitors")
    
    traindf = visitor_history(traindf)
    print("cleaned visitor history")
    
    traindf = price_quality(traindf)
    print("cleaned price quality")
    
    traindf = price_category(traindf)
    print("cleaned price category")
    
    traindf = process_remaining_cols(traindf)
    print("did remaining columns")
    
     # Add relevant columns    
    if testing_set is False:

        df = traindf[["prop_id", "srch_id", "position", "price_quality", 
                  "competitor_lower", "competitor_available", "visited_before", 
                  "click_bool", "booking_bool", "PriceBand", 
                  "promotion_flag", "prop_brand_bool", "random_bool",
                  "prop_location_score1", "prop_location_score2"]]
        df = add_score(df)
        df.drop(columns=["click_bool", "booking_bool"])
    else:
        df = traindf[["prop_id", "srch_id", "price_quality", 
                  "competitor_lower", "competitor_available", "visited_before", 
                  "PriceBand", 
                  "promotion_flag", "prop_brand_bool", "random_bool",
                  "prop_location_score1", "prop_location_score2"]]
    
    return df

In [ ]:
# Oh jit kan niet met pandas Series werken, moet omgeschreven worden naar np arrays
# @jit(nopython=True)
df = preprocessing(traindf)

In [ ]:
display(df.head())
display(df.describe())

In [ ]:
# df["price_quality"] = df.price_quality.astype(np.float32)
print(df.dtypes)
print()

if df.isnull().sum().sum() != 0:    
    print("\x1b[31mMissing values: \'\x1b[0m")
    print(df.isnull().sum())
else:
    print("\x1b[31mNo missing values!! :D \'\x1b[0m")

## Totale dataset zonder missende waardes, hier kan RandomForestClassifier op gerund worden

In [ ]:
# save the dataframe if it does not exist yet
if not os.path.exists(filename):
    df.to_hdf(filename, key="df", format="table")

In [ ]:
# test if it worked
reread = pd.read_hdf(filename)

In [ ]:
display(reread.describe())